In [1]:
#importing libraries
import nltk
import pandas as pd
#from emoticons import EmoticonDetector
import re as regex
import string
import collections
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.wsd import lesk
from nltk.corpus import wordnet
import operator
import numpy as np
from nltk.stem.snowball import SnowballStemmer
import plotly
from plotly import graph_objs
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
import time
import gensim
from gensim import corpora
import csv
import _pickle as cPickle
from nltk.corpus import words, stopwords

C:\Users\haris\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning:

detected Windows; aliasing chunkize to chunkize_serial



In [2]:
start_time = time.time()
num_sem_words = 13

kb = set(['bag.n.01','bag.n.04','bag.n.05','bag.n.06','baggage.n.01','baggage.n.03','staff.n.01', 'lost.a.01','dress.v.02','dress.v.10','dress.v.02','break.v.02','clothing.n.01','apparel.n.01','staff.v.02','seat.n.01','seat.v.01','seat.v.05','call.n.01','support.n.02','support.n.01','support.n.11','support.v.01','booking.n.02','reservation.n.06','avail.n.01','internet.n.01', 'engagement.n.05', 'booking.n.02', 'reserve.v.04', 'answer.v.01', 'answer.v.02', 'answer.v.03', 'answer.v.05', 'answer.v.08', 'answer.v.09', 'answer.v.10',  'answer.n.01', 'solution.n.02', 'answer.n.03', 'answer.n.05', 'service.n.01', 'service.n.02', 'service.n.05', 'avail.n.01', 'overhaul.n.01', 'service.n.15', 'call.n.01', 'cry.n.01', 'call.n.06', 'call.n.09', 'name.v.01', 'call.v.03', 'shout.v.02', 'call.v.05', 'visit.v.03', 'call.v.07', 'call.v.09', 'call.v.10', 'call.v.11', 'address.v.06', 'bid.v.04', 'call.v.23', 'call.v.24', 'call.v.25', 'telephone.n.01', 'earphone.n.01', 'chat.n.01', 'support.n.01', 'support.n.02', 'web_site.n.01', 'table.n.02', 'table.n.03', 'floor.n.01', 'floor.n.06', 'shock.v.01', 'customer.n.01', 'services.n.01', 'service.v.01', 'service.v.02', 'speak.v.03', 'address.v.02', 'reservation.n.01', 'reservation.n.05', 'reservation.n.06', 'reservation.n.07', 'hang.v.01', 'hang.v.02', 'hang.v.04', 'hang.v.08', 'cling.v.03', 'ticket.n.01', 'tag.n.01', 'ticket.v.02', 'class.n.01', 'class.n.03', 'class.n.08', 'classify.v.01', 'web.n.01', 'network.n.01', 'world_wide_web.n.01', 'reply.n.02', 'aid.n.02', 'talk.v.02', 'talk.v.01', 'communication.n.01', 'agent.n.02', 'message.n.01', 'message.v.03', 'staff.n.01', 'fee.n.01', 'service.n.02', 'electronic_mail.n.01', 'e-mail.v.01', 'on-line.a.02', 'help.v.01', 'desk.n.01', 'password.n.01', 'card.n.02', 'error.n.06', 'browser.n.02'])

df = pd.read_csv('../data/Bag.csv')
df2 = pd.read_csv('../data/Flight.csv')
df3 = pd.read_csv('../data/Service.csv')
#df4 = pd.DataFrame()
#df5 = pd.DataFrame()
#df6 = pd.DataFrame()
#dfannon = pd.read_csv('Annoted_Training_Tweets_13_semwords.csv')
#df_input = pd.read_csv('Input_Tweets.csv')
df_senti = pd.DataFrame()
#df_inter = pd.DataFrame()
df_inter = pd.read_csv('../data/Interim_7_words.csv')
df_clean = pd.DataFrame()
#df_clean = pd.read_csv('Interim_clean.csv')


df_Bag = df[:630]
df_Flight = df2[:2615]
df_Bag_test = df[630:]
df_Flight_test = df2[2615:]
len_df3 = int(0.8*float(len(df3['text'].tolist())))
df_Service = df3[:len_df3]
df_Service_test = df3[len_df3:]

list_text = []
list_topic = []
# list_sentiment = []
# list_text_Bag = []
# list_text_Flight = []
# list_text_Service = []

# training_tweets = df_Bag['text'].tolist()
# training_tweets.extend(df_Flight['text'].tolist())
# training_tweets.extend(df_Service['text'].tolist())
training_tweets = df_inter['text'].tolist()

##testing_tweets = df_Bag_test['text'].tolist()
##testing_tweets = df_input['text'].tolist()
testing_tweets = []

topics = ['bag','flight']

set_of_adv = set()


In [3]:
def strip_all_entities(text):
    entity_prefixes = ['@', '#', 'virgin', 'america', 'united', 'delta', 'jetblue', 'southwest']
    for separator in string.punctuation:
        if separator not in entity_prefixes:
            text = text.replace(separator, " ")
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                # words.append(" ")
                words.append(word)
            # print(type(''.join(words)))
    return " ".join(words)



In [4]:
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

In [5]:
def clean(doc):
    try:
        stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
        punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
        normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
        return normalized
    except:
        pass

doc_clean = [clean(tweet).split() for tweet in training_tweets]

In [6]:
print("--- %s seconds ---" % (time.time() - start_time))

dictionary = corpora.Dictionary(doc_clean)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

# Lda = gensim.models.ldamodel.LdaModel
# ldamodel = Lda(doc_term_matrix, num_topics=3, id2word = dictionary, passes=250)

ldamodel = gensim.models.LdaModel.load('../model/Model_phase_3_7_words')
print(ldamodel.print_topics(num_topics=3, num_words=9))

dict_topic_number = {}
dict_prediction = {}
dict_prediction[0] = 0
dict_prediction[1] = 0
dict_prediction[2] = 0

temp_set = set([0,1,2])
for tweet in topics:
    doc = clean(tweet)
    bow = dictionary.doc2bow(doc.split())
    t = ldamodel.get_document_topics(bow)
    maxi = 0.0
    tid = -1
    for ele in t:
        if ele[1]>maxi:
            maxi = ele[1]
            tid = ele[0]
    dict_topic_number[int(tid)] = tweet
    temp_set.remove(int(tid))

temp_tid = int(temp_set.pop())
dict_topic_number[temp_tid] = 'service'
print(dict_topic_number)

print("--- %s seconds ---" % (time.time() - start_time))
print("Enter number of testing tweets")
number = int(input())
for i in range(number):
    print("Enter input tweet")
    #find sentiment of each tweet here, store in sentiment_value
    tweet = input()

    tweet = strip_all_entities(tweet)
    tokenized = nltk.word_tokenize(tweet)
    tagged = nltk.pos_tag(tokenized)

    set_of_n = set()
    set_of_v = set()
    set_of_adj = set()
    sync_set = set()
    for element in tagged:
        if element[1] == 'NN' or element[1] == 'NNS' or element[1] == 'NNP' or element[1] == 'NNPS':
            set_of_n.add(element[0])
        if element[1] == 'VB' or element[1] == 'VBD' or element[1] == 'VBG' or element[1] == 'VBN' or element[
            1] == 'VBP' or element[1] == 'VBZ':
            set_of_v.add(element[0])
        if element[1] == 'JJ' or element[1] == 'JJR' or element[1] == 'JJS':
            set_of_adj.add(element[0])
        if element[1] == 'RBR' or element[1] == 'RB' or element[1] == 'RBS' or element[1] == 'WRB':
            set_of_v.add(element[0])

    for each_ele in set_of_n:
        ele_to_add = lesk(tweet, each_ele, 'n')
        if ele_to_add is not None:
            sync_set.add(ele_to_add)
    for each_ele in set_of_v:
        ele_to_add = lesk(tweet, each_ele, 'v')
        if ele_to_add is not None:
            sync_set.add(ele_to_add)
    for each_ele in set_of_adj:
        ele_to_add = lesk(tweet, each_ele, 'a')
        if ele_to_add is not None:
            sync_set.add(ele_to_add)

    for each_know in kb:
        sync_each_know = wordnet.synset(each_know)
        for each_sync in sync_set:
            similarity = each_sync.wup_similarity(sync_each_know)
            if similarity is not None and float(similarity)>0.80:
                dotted_word = each_sync.name()
                temp_string = ""
                for j in range(len(dotted_word)):
                    if dotted_word[j] == '.':
                        temp_string += " " + dotted_word[:j]
                        break
                tweet += temp_string
    doc = clean(tweet)
    bow = dictionary.doc2bow(doc.split())
    t = ldamodel.get_document_topics(bow)
    maxi = 0.0
    tid = -1
    for ele in t:
        if ele[1]>maxi:
            maxi = ele[1]
            tid = ele[0]
    dict_prediction[int(tid)] += 1
    topic_name = dict_topic_number[int(tid)]
    list_text.append(tweet)
    list_topic.append(topic_name)
    #list_sentiment.append(sentiment_value)
    print(topic_name)
    print("--- %s seconds ---" % (time.time() - start_time))


df_senti['text'] = pd.Series(list_text)
df_senti['topic'] = pd.Series(list_topic)
#df_senti['sentiment'] = pd.Series(list_sentiment)
df_senti.to_csv('../data/Output_Final_phase.csv')

#ldamodel.save('final_model_phase_3')
print("--- %s seconds ---" % (time.time() - start_time))

--- 3.246664047241211 seconds ---
[(0, '0.056*"class" + 0.047*"service" + 0.046*"support" + 0.041*"communication" + 0.033*"message" + 0.030*"aid" + 0.024*"flight" + 0.020*"error" + 0.017*"network"'), (1, '0.050*"answer" + 0.041*"communication" + 0.037*"message" + 0.032*"error" + 0.031*"reply" + 0.031*"aid" + 0.028*"support" + 0.025*"flight" + 0.025*"solution"'), (2, '0.087*"bag" + 0.046*"web" + 0.043*"clothing" + 0.030*"baggage" + 0.022*"hang" + 0.019*"apparel" + 0.018*"customer" + 0.016*"floor" + 0.016*"united"')]
{2: 'bag', 1: 'flight', 0: 'service'}
--- 3.626708507537842 seconds ---
Enter number of testing tweets
3
Enter input tweet
why is my flight late?
flight
--- 14.088988304138184 seconds ---
Enter input tweet
@VirginAmerica this serivce of yours is very poor
service
--- 26.621584177017212 seconds ---
Enter input tweet
I found a bag someone lost
bag
--- 36.72991967201233 seconds ---
--- 36.73590302467346 seconds ---


In [7]:
class TwitterData_Initialize():
    processed_Traindata = []
    wordlist = []  
    data_model = None
    data_labels = None
    
    def initialize(self, csv_file, from_cached=None):
        if from_cached is not None:
            self.data_model = pd.read_csv(from_cached)
            return
        self.processed_Traindata = pd.read_csv(csv_file, usecols=[0,1])
        self.wordlist = []
        self.data_model = None
        self.data_labels = None

In [8]:
#Data Pre-processing for Sentiment Analysis
data = TwitterData_Initialize()
data.initialize("../data/Output_Final_Phase.csv")
data.processed_Traindata

,Unnamed: 0,text
0,0,why is my flight late
1,1,this serivce of yours is very poor
2,2,I found a bag someone lost


In [9]:
#Data Preprocessing
nltk.download('words')
word_dictionary = list(set(words.words()))

for alphabet in "bcdefghjklmnopqrstuvwxyzBCDEFGHJKLMNOPQRSTUVWXYZ":
    word_dictionary.remove(alphabet)
class DataPreprocessing:
    def iterate(self):
        for preprocessingMethod in [self.replaceProcessedHashtags,
                                   self.removeUrls,
                                   self.removeUsernames,
                                   self.removeElongatedWords,
                                   self.removeNa,
                                   self.replaceSlangWords,
                                   self.removeSpecialChars,
                                   self.removeNumbers]:
            yield preprocessingMethod
    
    @staticmethod
    def removeByRegex(tweets, regExp):
        tweets.loc[:, "text"].replace(regExp, "", inplace=True)
        return tweets
    
    def removeUrls(self, tweets):
        return DataPreprocessing.removeByRegex(tweets, regex.compile(r"http.?://[^\s]+[\s]?"))
    
    def removeNa(self, tweets):
        return tweets[tweets["text"] != ""]
    
    def removeSpecialChars(self, tweets):
        for remove in map(lambda r: regex.compile(regex.escape(r)), [",", ":", "\"", "=", "&", ";", "%", "$"
                                                                    "@", "%", "^", "*", "(", ")", "{", "}",
                                                                    "[", "]", "|", "/", "\\", ">", "<", "-",
                                                                    "!", "?", ".", "'",
                                                                    "--", "---","#"]):
            tweets.loc[:, "text"].replace(remove, "", inplace=True)
        return tweets
    
    def removeUsernames(self, tweets):
        return DataPreprocessing.removeByRegex(tweets, regex.compile(r"@[^\s]+[\s]?"))
    
    def removeElongatedWords(self, tweets):
        return DataPreprocessing.removeByRegex(tweets, regex.compile(r"(.)\1+', r'\1\1"))
    
    def removeNumbers(self, tweets):
        #print(tweets)
        return DataPreprocessing.removeByRegex(tweets, regex.compile(r"\s?[0-9]+\.?[0-9]*"))
    
    def replaceSlangWords(self, tweets):
        with open('../data/slang.txt') as file:
            slang_map = dict(map(str.strip, line.partition('\t')[::2])
            for line in file if line.strip())
            #print(tweets["text"])
            #print("-----------------------------------------END")
            for index,word in tweets['text'].iteritems():
                #print(index)
                for i in word.split():
                    isUpperCase = i.isupper()
                    i = i.lower()
                    if i in slang_map.keys():
                        word = word.replace(i, slang_map[i])
                        tweets.loc[(index),"text"] = word
                if isUpperCase:
                    i = i.upper()
        #print(tweets.loc[:,"text"])
        return tweets

    # print(split_tweets)
    @staticmethod
    def removeDigitsFromHashtag(tag):
        tag = regex.sub(r"\s?[0-9]+\.?[0-9]*", "", tag)
        return tag

    @staticmethod
    def collect_hashtags_in_tweet(wordList):
        hashtags = []
        for word in wordList:
            index = word.find('#')
            if index != -1:
                if word[index + 1:] != '':
                    hashtags.append(word[index + 1:])
        return hashtags

    @staticmethod
    def split_hashtag_to_words_all_possibilities(hashtag):
        all_possibilities = []

        split_posibility = [hashtag[:i] in word_dictionary for i in reversed(range(len(hashtag) + 1))]
        possible_split_positions = [i for i, x in enumerate(split_posibility) if x == True]

        for split_pos in possible_split_positions:
            split_words = []
            word_1, word_2 = hashtag[:len(hashtag) - split_pos], hashtag[len(hashtag) - split_pos:]

            if word_2 in word_dictionary:
                split_words.append(word_1)
                split_words.append(word_2)
                all_possibilities.append(split_words)

                another_round = DataPreprocessing.split_hashtag_to_words_all_possibilities(word_2)

                if len(another_round) > 0:
                    all_possibilities = all_possibilities + [[a1] + a2 for a1, a2, in
                                                             zip([word_1] * len(another_round), another_round)]
            else:
                another_round = DataPreprocessing.split_hashtag_to_words_all_possibilities(word_2)

                if len(another_round) > 0:
                    all_possibilities = all_possibilities + [[a1] + a2 for a1, a2, in
                                                             zip([word_1] * len(another_round), another_round)]

        return all_possibilities

    @staticmethod
    def process_all_hashtags_in_tweet(hashtags):
        all_words = []
        for tag in hashtags:
            split_hashtag = DataPreprocessing.split_hashtag_to_words_all_possibilities(DataPreprocessing.removeDigitsFromHashtag(tag))
            if split_hashtag:
                all_words = all_words + split_hashtag[0]
            else:
                all_words.append(tag)
        return all_words 
    
    def replaceProcessedHashtags(self, tweets):
            for index,word in tweets['text'].iteritems():
                word=word.split()
                collectHashtags=DataPreprocessing.collect_hashtags_in_tweet(word)
                allHashtags=DataPreprocessing.process_all_hashtags_in_tweet(collectHashtags)
                collectHashtags = ["#" + tag for tag in collectHashtags]
                if allHashtags:
                    word = list(set(word) - set(collectHashtags))
                    word = word + allHashtags
                word = " ".join(word)
                tweets.loc[(index),"text"] = word
                #print(tweets.loc[(index),"text"])
           # print(tweets)
            return tweets

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\haris\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [10]:
#Cleaning the Training Data
class CleanTrainingData(TwitterData_Initialize):
    def __init__(self, previous):
        self.processed_Traindata = previous.processed_Traindata
        #self.processed_Testdata = previous.processed_Testdata
    
    def cleaningData(self, cleaner):
        train = self.processed_Traindata
        #test = self.processed_Testdata
        
        for cleanerMethod in cleaner.iterate():
            train = cleanerMethod(train)
            #test = cleanerMethod(test)
        self.processed_Traindata = train
        #self.processed_Testdata = test

In [11]:
data = CleanTrainingData(data)
data.cleaningData(DataPreprocessing())
data.processed_Traindata.head(5)

,Unnamed: 0,text
0,0,why is my flight late
1,1,this serivce of yours is very poor
2,2,I found a bag someone lost


In [12]:
#Tokenizing and Stemming the data
class TokenizationStemming(CleanTrainingData):
    def __init__(self, previous):
        self.processed_Traindata = previous.processed_Traindata
        #self.processed_TestData = previous.processed_TestData
    
    def stem(self, stemmer = nltk.PorterStemmer()):
        def stemJoin(row):
            row["text"] = list(map(lambda str: stemmer.stem(str.lower()), row["text"]))
            return row
    
        self.processed_Traindata = self.processed_Traindata.apply(stemJoin, axis=1)
    
    def tokenize(self, tokenizer = nltk.word_tokenize):
        def tokenizeRow(row):
            row["text"] = tokenizer(row["text"])
            row["tokenizedText"] = [] + row["text"]
            return row
        
        self.processed_Traindata = self.processed_Traindata.apply(tokenizeRow, axis=1)

In [13]:
#Building Wordlist
#Un-filtered version without removing stopwords
words = collections.Counter()
for idx in data.processed_Traindata.index:
    words.update(data.processed_Traindata.loc[idx, "text"])

words.most_common(5)

[(' ', 15), ('o', 8), ('s', 7), ('e', 6), ('i', 5)]

In [14]:
#Removing stopwords
stopwords = nltk.corpus.stopwords.words("english")
whitelist = ["n't", "not"]
for idx, stop_word in enumerate(stopwords):
    if stop_word not in whitelist:
        del words[stop_word]

words.most_common(5)

[(' ', 15), ('e', 6), ('r', 4), ('h', 3), ('f', 3)]

In [15]:
#Generating the final wordlist
class WordList(TokenizationStemming):
    def __init__(self, previous):
        self.processed_Traindata = previous.processed_Traindata
    
    whitelist = ["n't", "not"]
    wordlist = []
    
    def buildWordlist(self, min_occurrences=3, max_occurences=3000, stopwords=nltk.corpus.stopwords.words("english"),
                     whitelist=None):
        self.wordlist = []
        whitelist = self.whitelist if whitelist is None else whitelist
        import os
        if os.path.isfile('../data/wordlist.csv'):
            word_df = pd.read_csv('../data/wordlist.csv', encoding = "ISO-8859-1")
            word_df = word_df[word_df["occurrences"] > min_occurrences]
            self.wordlist = list(word_df.loc[:, "word"])
            return
        words = collections.Counter()
        for idx in self.processed_Traindata.index:
            words.update(self.processed_Traindata.loc[idx, "text"])
        
        for idx, stop_word in enumerate(stopwords):
            if stop_word not in whitelist:
                del words[stop_word]
        
        word_df = pd.DataFrame(data={"word" : [k for k, v in words.most_common() if min_occurrences < v < max_occurences],
                                    "occurrences": [v for k, v in words.most_common() if min_occurrences < v < max_occurences]},
                              columns = ["word", "occurrences"])
        
        word_df.to_csv("../data/wordlist.csv", index_label="idx", encoding = "utf8")
        self.wordlist = [k for k, v in words.most_common() if min_occurrences < v < max_occurences]
    

In [16]:
data = WordList(data)
data.buildWordlist()

In [17]:
#Transforming into Bag-of-Words
class BagOfWords(WordList):
    def __init__(self, previous):
        self.processed_Traindata = previous.processed_Traindata
        self.wordlist = previous.wordlist
    
    def buildDataModel(self):
        columns = list(
            map(lambda w: w + "_bow", self.wordlist))
        labels = []
        rows = []
        
        for idx in self.processed_Traindata.index:
            currentRow = []
            
            tokens = set(self.processed_Traindata.loc[idx, "text"])
            for _, word in enumerate(self.wordlist):
                currentRow.append(1 if word in tokens else 0)
            
            rows.append(currentRow)
        
        self.data_model = pd.DataFrame(rows, columns=columns)
        self.data_labels = pd.Series(labels)
        
        return self.data_model, self.data_labels

In [18]:
data = BagOfWords(data)
bow, labels = data.buildDataModel()
bow.head(5)

,hour_bow,cancel_bow,help_bow,servic_bow,delay_bow,time_bow,custom_bow,bag_bow,call_bow,im_bow,...,gorgeou_bow,woohoo_bow,thousand_bow,understat_bow,furiou_bow,manual_bow,smell_bow,ber_bow,charleston_bow,nrt_bow
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
#Detecting Emoticons
class EmoticonDetector:
    emoticons = {}
    
    def __init__(self, emoticon_file="../data/emoticons.txt"):
        from pathlib import Path
        content = Path(emoticon_file).read_text()
        positive = True
        for line in content.split("\n"):
            if "positive" in line.lower():
                positive = True
                continue
            elif "negative" in line.lower():
                positive = False
                continue
            
            self.emoticons[line] = positive
    
    def is_positive(self, emoticon):
        if emoticon in self.emoticons:
            return self.emoticons[emoticon]
        return False
    
    def is_emoticon(self, to_check):
        return to_check in self.emoticons

In [20]:
class ExtraFeatures(WordList):
    def __init__(self):
        pass
    
    def build_data_model(self):
        extra_columns = [col for col in self.processed_Traindata.columns if col.startswith("number_of")]
        columns = extra_columns + list(
                map(lambda w: w + "_bow", self.wordlist))
        
        labels = []
        rows = []
        for idx in self.processed_Traindata.index:
            current_row = []
        
            for _,col in enumerate(extra_columns):
                current_row.append(self.processed_Traindata.loc[idx, col])
        
        #adding bag-of-words
            tokens = set(self.processed_Traindata.loc[idx, "text"])
            for _,word in enumerate(self.wordlist):
                current_row.append(1 if word in tokens else 0)
        
            rows.append(current_row)
        
        self.data_model = pd.DataFrame(rows, columns=columns)
        self.data_labels = pd.Series(labels)
        
        return self.data_model, self.data_labels
    
    def add_column(self, column_name, column_content):
        self.processed_Traindata.loc[:, column_name] = pd.Series(column_content, index=self.processed_Traindata.index)

    def build_features(self):
        def count_by_lambda(expression, word_array):
            return len(list(filter(expression, word_array)))
        
        def count_occurences(character, word_array):
            counter = 0
            for j, word in enumerate(word_array):
                for char in word:
                    if char == character:
                        counter += 1
            return counter
        
        def count_interjections(wordArray):
            interjections = []
            interjectionCount = 0
            with open('../data/interjections.txt') as file:
                interjections = file.read().splitlines()
            for word in wordArray:
                if word in interjections:
                    interjectionCount += 1
            return interjectionCount 

        def count_by_regex(regex, plain_text):
            return len(regex.findall(plain_text))
        
        self.add_column("splitted_text", map(lambda txt: txt.split(" "), self.processed_Traindata["text"]))
        
        #Number of uppercase words
        uppercase = list(map(lambda txt: count_by_lambda(lambda word: word == word.upper(), txt),
                                                        self.processed_Traindata["splitted_text"]))
        self.add_column("number_of_uppercase", uppercase)
        
        #number of !
        exclamations = list(map(lambda txt: count_occurences("!", txt),
                               self.processed_Traindata["splitted_text"]))
        self.add_column("number_of_exclamation", exclamations)
        
        #number of ?
        questions = list(map(lambda txt: count_occurences("?", txt),
                               self.processed_Traindata["splitted_text"]))
        self.add_column("number_of_question", questions)
        
        #number of ...
        ellipsis = list(map(lambda txt: count_by_regex(regex.compile(r"\.\s?\.\s?\."), txt),
                           self.processed_Traindata["text"]))
        self.add_column("number_of_ellipsis", ellipsis)
        
        #number of hashtags
        hashtags = list(map(lambda txt: count_occurences("#", txt),
                               self.processed_Traindata["splitted_text"]))
        self.add_column("number_of_hashtags", hashtags)
        
        #number of mentions
        mentions = list(map(lambda txt: count_occurences("@", txt),
                               self.processed_Traindata["splitted_text"]))
        self.add_column("number_of_mentions", mentions)
        
        #number of quotes
        quotes = list(map(lambda plain_text: int(count_occurences("'", [plain_text.strip("'").strip('"')]) / 2 +
                                                 count_occurences('"', [plain_text.strip("'").strip('"')]) / 2),
                          self.processed_Traindata["text"]))
        self.add_column("number_of_quotes", quotes)
        
        #number of urls
        urls = list(map(lambda txt: count_by_regex(regex.compile(r"http.?://[^\s]+[\s]?"), txt),
                             self.processed_Traindata["text"]))
        self.add_column("number_of_urls", urls)
        
        #number of positive emoticons
        ed = EmoticonDetector()
        positive_emo = list(
            map(lambda txt: count_by_lambda(lambda word: ed.is_emoticon(word) and ed.is_positive(word), txt), 
                   self.processed_Traindata["splitted_text"]))
        self.add_column("number_of_positive_emo", positive_emo)
        
        #number of negative emoticons
        negative_emo = list(
            map(lambda txt: count_by_lambda(lambda word: ed.is_emoticon(word) and not ed.is_positive(word), txt), 
                   self.processed_Traindata["splitted_text"]))
        self.add_column("number_of_negative_emo", negative_emo)
        
        #number of interjections
        interjections = list(map(lambda txt: count_interjections(txt),
                               self.processed_Traindata["splitted_text"]))
        self.add_column("number_of_interjections", interjections)

In [21]:
data = ExtraFeatures()
data.initialize("../data/Output_Final_Phase.csv")
data.build_features()
data.cleaningData(DataPreprocessing())
data.tokenize()
data.stem()
data.buildWordlist()
data_model, labels = data.build_data_model()

data_model

,number_of_uppercase,number_of_exclamation,number_of_question,number_of_ellipsis,number_of_hashtags,number_of_mentions,number_of_quotes,number_of_urls,number_of_positive_emo,number_of_negative_emo,...,gorgeou_bow,woohoo_bow,thousand_bow,understat_bow,furiou_bow,manual_bow,smell_bow,ber_bow,charleston_bow,nrt_bow
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
#Load Naive-Bayes Library
with open('../model/NaiveBayesClassifier.pkl', 'rb') as nid:
    nb_loaded = cPickle.load(nid)
with open('../model/RandomForestClassifier.pkl', 'rb') as rid:
    rf_loaded = cPickle.load(rid)

result_nb = nb_loaded.predict(data_model)
print(type(result_nb))
print("Naive-Bayes Prediction : ",result_nb)
result_rf = rf_loaded.predict(data_model)
print("Random Forest Prediction : ",result_rf)
df_csv = pd.read_csv("../data/Output_Final_Phase.csv")
df_csv['NaiveBayesSentiment'] = pd.DataFrame(result_nb)
df_csv['RandomForestSentiment'] = pd.DataFrame(result_rf)

df_csv.to_csv("../data/Output_Final_Phase.csv")

ValueError: Expected input with 2424 features, got 2425 instead